# 5. Baseline EMA Strategy with Regime Filter
## 5/15 EMA Crossover Strategy Implementation and Backtesting

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_utils import load_data
from strategy import EMAStrategy, TradeAnalyzer
from backtest import Backtester, BacktestVisualizer, generate_backtest_report, save_backtest_results
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

In [ ]:
# Load data with regimes
df = load_data('../data/nifty_with_regimes.csv')
print(f"Data shape: {df.shape}")
df.head()

## 5.1 Split Data (Training: 70%, Testing: 30%)

In [ ]:
# Split data
backtester = Backtester(initial_capital=100000, commission=0.0003)
train_df, test_df = backtester.split_data(df, train_size=0.7)

print(f"Training set: {len(train_df)} rows")
print(f"Testing set: {len(test_df)} rows")

## 5.2 Implement EMA Strategy

In [ ]:
# Initialize strategy
strategy = EMAStrategy(fast_ema=5, slow_ema=15)

# Generate signals with regime filter
train_df = strategy.generate_signals(train_df, use_regime_filter=True)
test_df = strategy.generate_signals(test_df, use_regime_filter=True)

# Generate positions
train_df = strategy.generate_positions(train_df)
test_df = strategy.generate_positions(test_df)

print("\nTraining Set Signals:")
print(f"Long signals: {np.sum(train_df['position'] == 1)}")
print(f"Short signals: {np.sum(train_df['position'] == -1)}")
print(f"Flat periods: {np.sum(train_df['position'] == 0)}")

print("\nTesting Set Signals:")
print(f"Long signals: {np.sum(test_df['position'] == 1)}")
print(f"Short signals: {np.sum(test_df['position'] == -1)}")
print(f"Flat periods: {np.sum(test_df['position'] == 0)}")

## 5.3 Backtest on Training Set

In [ ]:
# Run backtest on training data
train_results = backtester.backtest(train_df, strategy_name="EMA 5/15 with Regime Filter (Train)")

print("\nTraining Set Performance:")
print("=" * 80)
metrics = train_results['metrics']
print(f"Total Return: {metrics['total_return']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"Win Rate: {metrics['win_rate']:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
print(f"Total Trades: {metrics['total_trades']}")

## 5.4 Backtest on Testing Set

In [ ]:
# Run backtest on testing data
test_results = backtester.backtest(test_df, strategy_name="EMA 5/15 with Regime Filter (Test)")

print("\nTesting Set Performance:")
print("=" * 80)
metrics = test_results['metrics']
print(f"Total Return: {metrics['total_return']:.2f}%")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"Calmar Ratio: {metrics['calmar_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"Win Rate: {metrics['win_rate']:.2f}%")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")
print(f"Total Trades: {metrics['total_trades']}")

## 5.5 Visualize Results

In [ ]:
# Calculate returns for visualization
test_df_with_returns = backtester.calculate_returns(test_df)

# Plot equity curve
BacktestVisualizer.plot_equity_curve(
    test_df_with_returns,
    save_path='../plots/baseline_equity_curve.png'
)
print("Equity curve plot saved")

In [ ]:
# Plot drawdown
BacktestVisualizer.plot_drawdown(
    test_df_with_returns,
    save_path='../plots/baseline_drawdown.png'
)
print("Drawdown plot saved")

In [ ]:
# Plot returns distribution
BacktestVisualizer.plot_returns_distribution(
    test_df_with_returns,
    save_path='../plots/baseline_returns_distribution.png'
)
print("Returns distribution plot saved")

In [ ]:
# Plot trade analysis
if test_results['trades'] is not None and len(test_results['trades']) > 0:
    BacktestVisualizer.plot_trade_analysis(
        test_results['trades'],
        save_path='../plots/baseline_trade_analysis.png'
    )
    print("Trade analysis plot saved")

## 5.6 Generate Reports

In [ ]:
# Generate backtest report
generate_backtest_report(
    test_results,
    output_path='../results/baseline_strategy_report.txt'
)
print("Backtest report generated")

# Display report
with open('../results/baseline_strategy_report.txt', 'r') as f:
    print(f.read())

In [ ]:
# Save detailed results
save_backtest_results(test_results, '../results/baseline_strategy')
print("Detailed results saved to results/baseline_strategy_*")

## Summary

In [ ]:
print("=" * 80)
print("BASELINE STRATEGY SUMMARY")
print("=" * 80)
print(f"\nStrategy: 5/15 EMA Crossover with Regime Filter")
print(f"Training Period: {len(train_df)} candles (70%)")
print(f"Testing Period: {len(test_df)} candles (30%)")
print(f"\nTest Set Results:")
print(f"  Total Return: {test_results['metrics']['total_return']:.2f}%")
print(f"  Sharpe Ratio: {test_results['metrics']['sharpe_ratio']:.2f}")
print(f"  Win Rate: {test_results['metrics']['win_rate']:.2f}%")
print(f"  Total Trades: {test_results['metrics']['total_trades']}")
print("\nNext Step: Proceed to 06_ml_models.ipynb")